# Extended Kalman-filter

This is an implementation of a Kalman-filter and the extendedn Kalman-filter as aprt if an exercise of *Recursive Estimation*.

### Introduction

We model a discrete-time non-linear temporal system as:

\begin{align}
    \mathbf{x}(k) &= q_{k-1}(\mathbf{x}(k-1), \mathbf{v}(k-1) ), \\
    \mathbf{y}(k) &= h_k(\mathbf{x}(k), \mathbf{w}(k)), 
\end{align}

where $\mathbf{x}_k$ are latent states, $\mathbf{y}_k$ are  observations and $q$ and $h$ are (possibly non-linear) functions for states and observations at time $k \in \{0, 1, 2, \dots \}$.

We estimate the latent states using *Bayesian tracking* which usually consists of two steps:

* Prior update/state prediction: $f_p = f_{x(k) | z(k-1)} $
* Posterior update/filtering: $f_m = f_{x(k) | z(k) } $

For Gaussian error terms $\mathbf{v} \sim \mathcal{N}(\boldsymbol \mu_v, \boldsymbol \Sigma_v)$, $\mathbf{w} \sim \mathcal{N}(\boldsymbol \mu_w, \boldsymbol \Sigma_w)$ and *non-linear* $q$ and $h$, we can use an *extended Kalman filter* for estimation of the latent states, since then the state estimates are also Gaussian $\mathbf{x}(k) = \mathbf{x}_p(k) \sim \mathcal{N}(\hat{\boldsymbol \mu}_p(k), \hat{\boldsymbol \Sigma}_p(k))$.

The main difference to a regular Kalman filter is that we first do a linear approximation of the system equations at the current state estimate and then apply the regular Kalman filter updates for means and covariances.

First we load some `packages`:

In [1]:
from scipy.stats import norm, uniform, multivariate_normal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.mlab as mlab

In this exercise describe the following non-linear system:

\begin{align}
    \mathbf{x}(k) &= \sin \left( \alpha \mathbf{x}(k - 1) + \alpha \mathbf{v}(k-1)   \right), \\ 
    \mathbf{y}(k) &= \cos \left( \beta \mathbf{x}(k) \right) + k \mathbf{w}(k),
\end{align}

where $\alpha = 2 \pi$, $\beta = \pi$, $\mathbf{v} \sim \mathcal{N}(\mathbf{0}, \frac{1}{4}\mathbf{I}) $ and $\mathbf{w} \sim \mathcal{N}(\mathbf{0}, \frac{1}{4}\mathbf{I}) $. The initial state is $\mathbf{x}(0) = \begin{pmatrix}1/12 \\  1/6\end{pmatrix}$.

In [2]:
x0    = np.array([1/12, 1/6])
mu_m  = x0
cov_m = np.matrix([[0, 0], [0, 0]])

In [3]:
mu_v  = np.array([0, 0])
cov_v = np.matrix([[1/4, 0], [0, 1/4]])
mu_w  = np.array([0, 0])
cov_w = np.matrix([[1/4, 0], [0, 1/4]])

In [4]:
k = 1
alpha = 2 * np.pi
beta  = np.pi

In [28]:
def process_model(x, v, alpha):
    su = np.add(np.multiply(alpha, x), np.multiply(alpha, v))
    return np.sin(su)

def observation_model(x, w, k, beta):
    return np.add(np.cos(np.multiply(beta, x)), np.multiply(k, w))

#### Prior update

**Assuming $\mathbb{E}[\mathbf{v}(k−1)] = \mathbf{0} $** (otherwise we just substract the mean) we do a linear approximation of $q_{k-1}$ at $\mathbf{x}(k)$:

\begin{align}
    \mathbf{x}(k) &  \approx \ q_{k-1}(\hat{\boldsymbol \mu}_m(k-1), \mathbf{0}) 
     +  \mathbf{A}(k-1) (\mathbf{x}(k-1) - \hat{\boldsymbol \mu}_m(k-1)) 
     +  \mathbf{L}(k-1) \mathbf{v}(k-1) \\
     & = \mathbf{A}(k-1) \mathbf{x}(k-1) + \underbrace{ \mathbf{L}(k-1) \mathbf{v}(k-1) }_{ \tilde{ \mathbf{v} }(k-1) } + \underbrace{ q_{k-1}(\hat{\boldsymbol \mu}_m (k-1), \mathbf{0}) - \mathbf{A}(k-1) \hat{\boldsymbol \mu}_m (k-1) }_{ \boldsymbol \xi(k-1) } \\
     & = \mathbf{A}(k-1) \mathbf{x}(k-1) + \tilde{\mathbf{v}}(k-1) + \boldsymbol \xi(k-1),     
\end{align}

where $ \mathbf{A}(k-1) = \frac{\partial q_{k-1} (\hat{\boldsymbol \mu}_m(k-1), \mathbf{0}) }{\partial \mathbf{x}} $ and $ \mathbf{L}(k-1) = \frac{\partial q_{k-1} (\hat{\boldsymbol \mu}_m(k-1), \mathbf{0}) }{\partial \mathbf{v}} $.

The process noise has zero mean $\mathbb{E}[\mathbf{\tilde{v}}(k-1)] = \mathbf{0}$ and variance 
$\mathbb{V}[ \mathbf{\tilde{v}}(k-1) ] = \mathbf{L}(k-1) \boldsymbol \Sigma_v(k-1) \mathbf{L}^T(k-1)$.

So in our example we have the following equations:

\begin{align}
  \mathbf{A}(k) & = \alpha  \cos \left( \alpha \mathbf{x}(k) + \alpha \mathbf{v}(k)   \right) \\
  \mathbf{L}(k) & = \alpha \cos \left( \alpha \mathbf{x}(k) + \alpha \mathbf{v}(k)  \right) 
\end{align}

In [6]:
def a_k(alpha, x, v):
    su = np.multiply(alpha, x) + np.multiply(alpha, v)
    return np.diag(np.multiply(alpha, np.cos(su)))

def l_k(alpha, x, v):
    su = np.multiply(alpha, x) + np.multiply(alpha, v)
    return np.diag(np.multiply(alpha, np.cos(su)))

In [7]:
A = a_k(alpha, mu_m, mu_v)
print(A)
L = l_k(alpha, mu_m, mu_v)
print(L)

[[ 5.44139809  0.        ]
 [ 0.          3.14159265]]
[[ 5.44139809  0.        ]
 [ 0.          3.14159265]]


Then the prior update equation for the mean $\hat{\boldsymbol \mu}_p(k)$ is:

\begin{align}
 \hat{\boldsymbol \mu}_p(k) & = \mathbf{A}(k-1) \hat{\boldsymbol \mu}_m(k-1)  + \boldsymbol \xi(k-1) \\ 
 & = q_{k-1}(\hat{\boldsymbol \mu}_m(k-1), \mathbf{0}).
\end{align}

In [8]:
def update_prior_mean(process_model, mu_m, v, alpha):
    return process_model(mu_m, v, alpha)

In [9]:
mu_p = update_prior_mean(process_model, mu_m, mu_v, alpha)
print(mu_p)

[ 0.5        0.8660254]


And the update for the prior variance $\hat{\boldsymbol \Sigma}_p(k)$ is:

\begin{align}
 \hat{\boldsymbol \Sigma}_p(k) & = \mathbf{A}(k-1) \hat{\boldsymbol \Sigma}_m(k-1) \mathbf{A}^T(k-1) + \mathbf{L}(k-1) \boldsymbol \Sigma_v(k-1) \mathbf{L}^T(k-1).
\end{align}

In [10]:
def update_prior_covariance(cov_m, cov_v, A, L):
    su1 = np.matmul(np.matmul(A, cov_m), A.T)
    su2 = np.matmul(np.matmul(L, cov_v), L.T)
    return np.add(su1, su2)

In [11]:
cov_p = update_prior_covariance(cov_m, cov_v, A, L)
print(cov_p)

[[ 7.4022033  0.       ]
 [ 0.         2.4674011]]


#### Posterior update

We then do the posterior update by linearizing of $h_{k}$ at $\mathbf{x}(k)$. **We assume zero mean $\mathbb{E}[\mathbf{w}(k)] = \mathbf{0} $** for the measurement noise.

\begin{align}
 \mathbf{y}(k) &  \approx \ h_{k}(\hat{\boldsymbol \mu}_p(k), \mathbf{0}) 
     +  \mathbf{B}(k) (\mathbf{x}(k) - \hat{\boldsymbol \mu}_p(k)) 
     +  \mathbf{J}(k) \mathbf{w}(k) \\
     & = \mathbf{B}(k)\mathbf{x}(k) + \underbrace{\mathbf{J}(k)\mathbf{w}(k)}_{\tilde{\mathbf{w}}(k)} + 
      \underbrace{h_{k}(\hat{\boldsymbol \mu}_p(k), \mathbf{0})-  \mathbf{B}(k)\hat{\boldsymbol \mu}_p(k)}_{\boldsymbol \zeta(k)} \\
      & = \mathbf{B}(k)\mathbf{x}(k)+ \tilde{\mathbf{w}}(k) + \boldsymbol \zeta(k),
\end{align}

where $\mathbf{B}(k) = \frac{\partial h_{k} (\hat{\boldsymbol \mu}_p(k), \mathbf{0}) }{\partial \mathbf{x}}$ and $\mathbf{J}(k) = \frac{\partial h_{k} (\hat{\boldsymbol \mu}_p(k), \mathbf{0}) }{\partial \mathbf{w}}$.

The measurement noise has zero mean $\mathbb{E}[\mathbf{\tilde{w}}(k)] = \mathbf{0}$ and variance 
$\mathbb{V}[ \mathbf{\tilde{w}}(k) ] = \mathbf{J}(k) \boldsymbol \Sigma_w(k) \mathbf{J}^T(k)$.

So in our example we have the following equations:

\begin{align}
  \mathbf{B}(k) & = - \beta \sin \left( \beta \mathbf{x}(k) \right) \\ 
  \mathbf{J}(k) & = k
\end{align}

In [12]:
def b_k(beta, x):
    su = np.multiply(beta, x)
    return np.diag(np.multiply(beta, -np.sin(su)))

def j_k(k, dim):
    return np.diag([k] * dim)

In [13]:
B = b_k(beta, mu_p)
print(B)
J = j_k(k, 2)
print(J)

[[-3.14159265  0.        ]
 [ 0.         -1.28358009]]
[[1 0]
 [0 1]]


We first define the Kalman gain matrix at time $k$ as:

\begin{align}
    \mathbf{K}(k) &= \hat{\boldsymbol \Sigma}_p(k) \mathbf{B}^T(k) \left( \mathbf{B}(k) \hat{\boldsymbol \Sigma}_p(k) \mathbf{B}^T(k) + \mathbf{J}(k) \mathbf{\Sigma}_w(k) \mathbf{J}^T(k) \right)^{-1}.
\end{align}

In [14]:
def k_k(cov_p, cov_w, B, J):    
    cov_p_b = np.matmul(cov_p, B.T)
    cov_w_j = np.matmul(cov_w, J.T)
    within = np.add(np.matmul(B, cov_p_b), np.matmul(J, cov_w_j))
    return np.matmul(cov_p_b, np.linalg.inv(within))

In [15]:
K = k_k(cov_p, cov_w, B, J)
print(K)

[[-0.31722435  0.        ]
 [ 0.         -0.73393607]]


Then the update equation for the posterior mean $\hat{\boldsymbol \mu}_m(k)$ is:

\begin{align}
    \hat{\boldsymbol \mu}_m(k) & = \hat{\boldsymbol \mu}_p(k) + \mathbf{K}(k) \left( \mathbf{y}(k) - \mathbf{\Upsilon}(k) \hat{\boldsymbol \mu}_p(k) - \boldsymbol \zeta(k) \right) \\
     & = \hat{\boldsymbol \mu}_p(k) + \mathbf{K}(k) \left( \mathbf{y}(k) - h_{k}(\hat{\boldsymbol \mu}_p(k), \mathbf{0})\right).
\end{align}

In [16]:
def update_posterior_mean(y, x, w, k, beta, observation_model, mu_p, K):
    obs = observation_model(x, w, k, beta)
    su = np.dot(K, np.subtract(y, obs) - 2)
    return np.asarray(np.add(mu_p, su)).reshape(-1)

In [17]:
y = np.array([2, 2])
mu_m = update_posterior_mean(y, mu_p, mu_w, k, beta, observation_model, mu_p, K)
print(mu_m)

[ 0.5         0.19614419]


The update for the posterior covariance $\hat{\boldsymbol \Sigma}_m(k)$ is given by: 

\begin{align}
    \hat{\boldsymbol \Sigma}_m(k) &= \left( \mathbf{I} - \mathbf{K}(k) \mathbf{B}(k) \right)  \hat{\boldsymbol \Sigma}_p(k).
\end{align}

In [18]:
def update_posterior_covariance(K, B, cov_p):
    I = np.diag([1] * K.shape[0])
    su = np.subtract(I, np.matmul(K, B))
    return np.matmul(su, cov_p)

In [19]:
cov_m = update_posterior_covariance(K, B, cov_p)
print(cov_m)

[[ 0.02524391  0.        ]
 [ 0.          0.14294707]]
